# Week 8 - Neuromorphic computing - Exercise

Note: this is a new version of the exercise, for the old version see [w8-neuromorphic-exercise-v1.ipynb](w8-neuromorphic-exercise-v1.ipynb).

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neuro4ml/exercises/blob/main/w8-neuromorphic/w8-neuromorphic-exercise.ipynb)

## 🧠 Introduction 

Neuromorphic engineering is a field that aims to design and build artificial neural systems that mimic the architecture and principles of biological neural networks. Unlike traditional von Neumann computing architectures, neuromorphic chips:

1. 🔄 Process information in a parallel, event-driven manner
2. 💾 Integrate memory and computation
3. ⚡ Operate with extremely low power consumption

### 🤔 Why trade off power and accuracy?

Traditional deep learning models running on GPUs or CPUs consume significant power (often hundreds of watts). In contrast, the human brain processes complex information while consuming only ~20 watts. Neuromorphic chips aim to bridge this efficiency gap by:

- 📊 Using spike-based computation
- 🎯 Implementing local learning rules
- ⚡ Exploiting sparse, event-driven processing

However, these benefits often come with reduced accuracy compared to traditional deep learning approaches. Understanding and optimizing this trade-off is crucial for deploying neural networks in power-constrained environments like mobile devices or IoT sensors.

## 📝 Exercise overview

In this exercise, you will:
1. 🔧 Implement a simple neuromorphic chip simulator
2. 🏃‍♂️ Train SNNs with different architectures
3. 📊 Analyze the power-accuracy trade-off
4. 🔍 Explore how different parameters affect this trade-off

**This will also serve as a solid introduction on how to effectively train SNNs using modern packages such as SNNTorch!**

## 💻 Setup

Some of the code for this exercise is already provided, but you will need to implement some parts: 

### SNNModel (models.py)
The `SNNModel` class implements a 2-layer Leaky Integrate-and-Fire (LIF) network using SNNTorch. The network architecture consists of:
- Input layer → Hidden layer (with LIF neurons) → Output layer (with LIF neurons). (You will be able to play with other network architectures)
- Each LIF neuron has a decay rate (beta) that controls how quickly the membrane potential decays. (You will be able to play with other neuron models provided by SNNTorch)
- The network processes input data over multiple timesteps, producing spikes at each layer

### NeuromorphicChip (chip.py)
The `NeuromorphicChip` class simulates a neuromorphic hardware platform with the following constraints:
- Maximum number of neurons: 1024
- Maximum number of synapses: 64 * 1024
- Memory per neuron: 32 bytes
- Memory per synapse: 4 bytes
- Energy consumption:
  - 1e-1 nJ per neuron update
  - 5e-4 nJ per synapse event
  
This backend hardware is very simple and does not include many features of neuromorphic hardware, and serves only as an introduction to thinking about efficient network design.

## Imports and data loading

In [10]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install snntorch
    !git clone https://github.com/neuro4ml/exercises.git
    !cp exercises/w8-neuromorphic/*.py .
    !cp exercises/w8-neuromorphic/dataset .
    !cp exercises/w8-neuromorphic/dataset_labels .

# If you are using a local machine, please install the dependencies yourself.

In [4]:
# For automatic reloading of external modules
%load_ext autoreload
%autoreload 2

In [5]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

from chip import NeuromorphicChip
from models import SNNModel

## 🛠️ Exercise 1.1: Mapping Implementation

To complete this first question you need to implement the functions necessary to map your network on the chip.

- 📍 Go to [models.py](models.py) and implement the `n_neurons` and `n_synapses` properties.
- 📍 Go to [chip.py](chip.py) and implement the `calculate_memory_usage`, `map` and `run` methods.
- ▶️ Run the following cell to check your implementation

This is what you should see:

    Simulation Results:
    Energy consumption: 1.29 µJ
    Memory usage: 57.34 KB
    Total neuron updates: 11000
    Total synapse events: 389740
    Average spike rate: 0.205
    Total spikes: 3070.0

In [4]:
chip = NeuromorphicChip()

dims = (128, 100, 10)
n_timesteps = 100
seed = 42
snn = SNNModel(n_in=dims[0], n_hidden=dims[1], n_out=dims[-1], beta=0.95, seed=seed)

In [5]:
# Generate random input (seed is fixed to 42 for reproducibility)
torch.manual_seed(seed)
input_data = torch.randn(n_timesteps, dims[0]) * 10  # 100 timesteps

# Map the network on the chip
chip.map(snn)
# Run the network
output, results = chip.run(input_data=input_data)

print("\nSimulation Results:")
print(f"Energy consumption: {results['total_energy_nJ']/1000:.2f} µJ")
print(f"Memory usage: {results['memory_usage_bytes']/1024:.2f} KB")
print(f"Total neuron updates: {results['neuron_updates']}")
print(f"Total synapse events: {results['synapse_events']}")
print(f"Average spike rate: {results['spike_rate']:.3f}")
print(f"Total spikes: {results['total_spikes']}")


Simulation Results:
Energy consumption: 0.29 µJ
Memory usage: 57.34 KB
Total neuron updates: 110
Total synapse events: 553716
Average spike rate: 0.219
Total spikes: 4327.0


## 🚫 Exercise 1.2: Failed Mappings

Now let's explore what happens when we try to map networks that exceed the chip's constraints:

### 🔬 Experiments:
1. 🧠 First, we'll try mapping a network with too many neurons
2. 🔗 Then, we'll attempt to map one with too many synapses 
3. 💡 Finally, we'll see how sparse connectivity can help fit larger networks

Let's run these experiments and observe the error messages we get! Each case will demonstrate different limitations of neuromorphic hardware:
The first two cases should return a `MemoryError` if your code is correct. The third case should run without errors.


In [31]:
chip = NeuromorphicChip()

# Case 1 : Too many neurons
dims = (128, 1024, 10)
seed = 42
snn = SNNModel(n_in=dims[0], n_hidden=dims[1], n_out=dims[-1], beta=0.95, seed=seed)
# Map the network on the chip
try:
    chip.map(snn)
except MemoryError as e:
    print(e)

Too many neurons: 1034 (max: 1024)


In [32]:
chip = NeuromorphicChip()

# Case 2 : Too many synapses
dims = (128, 512, 10)
seed = 42
snn = SNNModel(n_in=dims[0], n_hidden=dims[1], n_out=dims[-1], beta=0.95, seed=seed)
# Map the network on the chip
try:
    chip.map(snn)
except MemoryError as e:
    print(e)

Too many synapses: 70656 (max: 65536)


In [33]:
# Case 3 : Sparse connectivity
dims = (128, 512, 10)
seed = 42
snn = SNNModel(n_in=dims[0], n_hidden=dims[1], n_out=dims[-1], beta=0.95, seed=seed)
for l in snn.layers:
    if hasattr(l, "weight"):
        l.weight.data = (
            torch.rand(l.weight.data.shape) < 0.5
        )  # 50% of the weights are non-zero

# Map the network on the chip
try:
    chip.map(snn)
    print(
        f"Mapped! Memory usage: {chip.calculate_memory_usage(snn)/1024:.2f} KB, Number of neurons: {snn.n_neurons}, Number of synapses: {snn.n_synapses}"
    )
except MemoryError as e:
    print(e)

Mapped! Memory usage: 154.16 KB, Number of neurons: 522, Number of synapses: 35289


## 🎯 Exercise 2: Training

In this exercise you will train a SNN on the [Randman dataset](https://github.com/fzenke/randman).

### 📊 Background: The Randman Dataset

The Randman dataset is a synthetic dataset specifically designed for training Spiking Neural Networks (SNNs). Here's what you need to know:

1. **Dataset Structure**
   - Generates labeled spike trains for classification
   - Each sample consists of temporal spike patterns
   - Data is organized into multiple classes (10 classes)
   - Spike times are stored in `dataset` file
   - Class labels are stored in `dataset_labels` file

2. **Data Format**
   - Input: Spike trains encoded as binary tensors (time x neurons)
   - Each neuron can spike at different time steps
   - Data is converted to one-hot encoding across time steps
   - Shape: (batch_size, timesteps, input_neurons)

3. **Classification Task**
   - Goal: Classify input spike patterns into correct classes
   - Output layer produces spike trains
   - Classification is done using rate coding (for now !): the output neuron that spikes the most indicates the predicted class

4. **Data Loading**
   All necessary code for loading and preprocessing the data is provided:
   - Data loading from files
   - Conversion to one-hot encoding
   - Train/test splitting
   - DataLoader creation with batching

### 🎓 2.1 Training

- 📝 Go to [training.py](training.py) and complete the `SNNTrainer` class, in particular the `calculate_accuracy` method
- ▶️ Run the following cell to train your network
- 📊 Take a look at the training and testing metrics, especially the accuracy and energy consumption
- 🔄 Start experimenting with different architectures and parameters to see how they affect the accuracy and energy consumption

In [2]:
from training import get_dataloaders, SNNTrainer

In [4]:
# Create dataloaders
train_loader, test_loader, dataset = get_dataloaders(
    batch_size=64,
)

torch.Size([64, 100, 128]) torch.Size([64])


In [5]:
# Take a look at the data
data, labels = next(iter(train_loader))
print(
    data.shape, labels.shape
)  # batch_size x timesteps x n_in. 1st and 2nd dims are swapped when passed to the model

torch.Size([64, 100, 128]) torch.Size([64])


In [6]:
snn_config = {
    "n_hidden": 128,
    "beta": 0.95,
    "seed": 42,
}

In [ ]:
# Initialize model
snn = SNNModel(
    n_hidden=snn_config["n_hidden"],
    beta=snn_config["beta"],
    seed=snn_config["seed"],
)

In [ ]:
# Initialize trainer
trainer = SNNTrainer(snn, learning_rate=1e-3, lr_gamma=0.9, config=snn_config)
# Train the model
trainer.train(train_loader, test_loader, n_epochs=10)

### 📈 2.2 Plot the results
- 📊 We can plot the accuracy and energy consumption as a function of the epoch
- 📈 We see that the accuracy is improving but the energy consumption is also increasing
- ⚖️ This is a trade-off that we need to be aware of when training SNNs

In [ ]:
results = trainer.pd_results.groupby("epoch", as_index=False).mean()
fig, ax = plt.subplots()
sns.lineplot(
    data=results, x="epoch", y="accuracy", ax=ax, label="Accuracy", legend=False
)
ax2 = ax.twinx()
sns.lineplot(
    data=results,
    x="epoch",
    y="total_energy_nJ",
    ax=ax2,
    color="orange",
    label="Energy",
    legend=False,
)
ax.figure.legend()
ax.set_title(
    f"Accuracy and Energy, Final Trade-off Score: {trainer.pareto_tradeoff:.2f}"
)
plt.show()

## 🚀 Exercise 3: Optimizing the trade-off

Now, you will explore how different parameters affect the accuracy and energy consumption of the SNN. This part is open-ended, here are some ideas:

-  Experiment with network architectures (number of layers, number of neurons, etc.)
-  Regularize spiking activity 
-  Implement a bi-exponential neuron model, using SnnTorch (snn.neurons.Synaptic)
- Implement a temporal loss (time-to-first-spike), using SnnTorch. Be careful to change the `calculate_accuracy` method in `training.py`
-  Implement weight masks to reduce the number of synapses
-  Use SnnTorch to make the time-constants heterogeneous and/or learnable, and maybe use less neurons

Ideally, after experimenting with these parameters, you should start to see a rough trade-off between accuracy and energy! Can we see some kind of Pareto front appearing? 

### 🏆 *The group with the best trade-off score will win the competition!*